# NN approaches to movie recommendation

## 1. Imports

In [39]:
# if needed
#pip install -q tensorflow-recommenders

In [45]:
# classic
import pandas as pd
import numpy as np
from datetime import datetime


## 2. Data 

### 2.1. MPST Dataset

source: https://www.kaggle.com/datasets/cryptexcode/mpst-movie-plot-synopses-with-tags

In [4]:
df_mpst = pd.read_csv('~/code/rossnorman11/raw_data/mpst/mpst_full_data.csv')

In [5]:
df.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


#### 2.1.1. Length of the plot synopsis

In [5]:
df_mpst['plot_word_count'] = df_mpst['plot_synopsis'].apply(lambda x: len(x.split()))

In [6]:
df_mpst.head()

,imdb_id,title,plot_synopsis,tags,split,synopsis_source,plot_word_count
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb,1315
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb,378
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb,707
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb,2317
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb,3063


### 2.2. MovieLens DB

source: https://kaggle.com/datasets/grouplens/movielens-20m-dataset/data

smaller: https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset/data

TO DO: Merging the separate tables we need into one csv and load it

For testing: Read the first 1000 rows of the dataset

df = pd.read_csv(file_path, nrows=1000)

#### 2.2.1. using the smaller db at first for NN testing: 

user reviews:

In [54]:
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_ml_user = pd.read_csv('~/code/rossnorman11/raw_data/movielens_100k/u.data', sep='\t', names=columns)
df_ml_user = df_ml_user.drop(columns='timestamp')

#df_ml_user['timestamp'] = df_ml_user['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

display(df_ml_user.head(5))
display(df_ml_user.describe())

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


,user_id,movie_id,rating
count,100000.00000,100000.000000,100000.000000
mean,462.48475,425.530130,3.529860
std,266.61442,330.798356,1.125674
min,1.00000,1.000000,1.000000
25%,254.00000,175.000000,3.000000
50%,447.00000,322.000000,4.000000
75%,682.00000,631.000000,4.000000
max,943.00000,1682.000000,5.000000


movie data:

In [22]:
columns = ['movie_id', 'movie_title', 'release date', 'video release date',
'IMDb URL',  'unknown', 'Action', 'Adventure',  'Animation',
'Childrens' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
'Film-Noir' , 'Horror' ,'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi',
'Thriller' , 'War' , 'Western']
df_ml_movie = pd.read_csv('~/code/rossnorman11/raw_data/movielens_100k/u.item', sep='|', names=columns, encoding="ISO-8859-1")
df_ml_movie = df_ml_movie[['movie_id', 'movie_title']]

display(df_ml_movie.head(5))
display(df_ml_movie.describe())

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


,movie_id
count,1682.000000
mean,841.500000
std,485.695893
min,1.000000
25%,421.250000
50%,841.500000
75%,1261.750000
max,1682.000000


mergin on movie_id:

In [55]:
df = df_ml_user.merge(df_ml_movie, how='left')

In [56]:
df.head()

,user_id,movie_id,rating,movie_title
0,196,242,3,Kolya (1996)
1,186,302,3,L.A. Confidential (1997)
2,22,377,1,Heavyweights (1994)
3,244,51,2,Legends of the Fall (1994)
4,166,346,1,Jackie Brown (1997)


In [40]:
df.rating.value_counts()

rating
4    34174
3    27145
5    21201
2    11370
1     6110
Name: count, dtype: int64

In [53]:
# just to see an example user
df[df['user_id'] == 196]

,user_id,movie_id,rating,timestamp,movie_title
0,196,242,3,1997-12-04 16:55:49,Kolya (1996)
940,196,393,4,1997-12-04 17:11:03,Mrs. Doubtfire (1993)
1133,196,381,4,1997-12-04 17:08:48,Muriel's Wedding (1994)
1812,196,251,3,1997-12-04 17:01:14,Shall We Dance? (1996)
1896,196,655,5,1997-12-04 17:09:53,Stand by Me (1986)
2374,196,67,5,1997-12-04 17:13:37,Ace Ventura: Pet Detective (1994)
6910,196,306,4,1997-12-04 16:57:01,"Mrs. Brown (Her Majesty, Mrs. Brown) (1997)"
7517,196,238,4,1997-12-04 17:10:20,Raising Arizona (1987)
7842,196,663,5,1997-12-04 17:11:51,Being There (1979)
10017,196,111,4,1997-12-04 17:09:53,"Truth About Cats & Dogs, The (1996)"


## 3. Collecting ideas for NN approach: TensorFlow Recomenders

### 3.1. What is my input, what is my prediction?

#### input: 

movie title(s?) from the preprocessing/ML step, which outputs a movie(/movies?) for a user prompt 

(see notebooks oscarjp7_preprocessing and Transformer_encoder)

#### output: 

movie recommendations for a user, based on his prompt

### 3.2. Which models/networks are suitable?

TensorFlow Recommenders: https://www.tensorflow.org/recommenders

also used on Kaggle, e.g.: https://www.kaggle.com/code/faiqueali/tensorflow-movie-recommender-system#Code 

or https://www.kaggle.com/code/lounessaidoun1010/basic-recommender-system

### Steps

a) data collection (see above); needed here: ratings and movie df

b) preprocessing (see preprocessing notebook) --> Pipeline architecture

c) embedding

d) hybrid model architecture: collaborative filtering (making recommendation based on previous behavior) + neural network 

e) training, predicting and evaluating

### 3.3. Creating two dataframes to train a model

In [80]:
# rating df
ratings = df[['user_id', 'rating', 'movie_title']]
ratings.head()


,user_id,rating,movie_title
0,196,3,Kolya (1996)
1,186,3,L.A. Confidential (1997)
2,22,1,Heavyweights (1994)
3,244,2,Legends of the Fall (1994)
4,166,1,Jackie Brown (1997)


In [81]:
ratings.dtypes

user_id         int64
rating          int64
movie_title    object
dtype: object

In [82]:
ratings['user_id'] = ratings['user_id'].astype(str)

/tmp/ipykernel_142120/1450787375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ratings['user_id'] = ratings['user_id'].astype(str)


In [84]:
movies = df[['movie_title']]
movies.head()

,movie_title
0,Kolya (1996)
1,L.A. Confidential (1997)
2,Heavyweights (1994)
3,Legends of the Fall (1994)
4,Jackie Brown (1997)


In [85]:
movies.dtypes

movie_title    object
dtype: object

### Converting pd.df to tf (needed for the TF Recommender)

In [86]:
dict(ratings)

{'user_id': 0        196
 1        186
 2         22
 3        244
 4        166
         ... 
 99995    880
 99996    716
 99997    276
 99998     13
 99999     12
 Name: user_id, Length: 100000, dtype: object,
 'rating': 0        3
 1        3
 2        1
 3        2
 4        1
         ..
 99995    3
 99996    5
 99997    1
 99998    2
 99999    3
 Name: rating, Length: 100000, dtype: int64,
 'movie_title': 0                        Kolya (1996)
 1            L.A. Confidential (1997)
 2                 Heavyweights (1994)
 3          Legends of the Fall (1994)
 4                 Jackie Brown (1997)
                      ...             
 99995    First Wives Club, The (1996)
 99996       Back to the Future (1985)
 99997                   Sliver (1993)
 99998           101 Dalmatians (1996)
 99999               Unforgiven (1992)
 Name: movie_title, Length: 100000, dtype: object}

In [75]:
dict(movies)

{'movie_title': 0                        Kolya (1996)
 1            L.A. Confidential (1997)
 2                 Heavyweights (1994)
 3          Legends of the Fall (1994)
 4                 Jackie Brown (1997)
                      ...             
 99995    First Wives Club, The (1996)
 99996       Back to the Future (1985)
 99997                   Sliver (1993)
 99998           101 Dalmatians (1996)
 99999               Unforgiven (1992)
 Name: movie_title, Length: 100000, dtype: object}

In [87]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))
movies = tf.data.Dataset.from_tensor_slices(dict(movies))

# 5. Evaluation of the model?

### Does the model make good recommendations?


### 4.1. train data: 
model learns patterns, which movies are connected to which "user-types" (with similar preferences)

learned connections:

    user(s) - movie(s) (who watches what?)

    movie(s) - user's review score(s) (how good was the rating for a movie?)

### 4.2. validation/test data: 

target: review score 

based on learned patterns: what would be the review score --> compare y_pred and y_true

--> possible to check, if a recommendation fits to a user

